In [63]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import statsmodels.api as sm
import math
import numpy as np
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [64]:
# Muat data dari file CSV dengan delimiter ;
df = pd.read_csv('merged_data.csv', delimiter=';')
df = df.drop('Price_y', axis=1)
df.head()

,TransactionID,CustomerID,Date,ProductID,Price_x,Qty,TotalAmount,StoreID,Age,Gender,Marital Status,Income,StoreName,GroupStore,Type,Latitude,Longitude,Product Name
0,TR11369,328,01/01/2022,P3,7500,4,30000,12,36,0,Married,"10,53",Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554",Crackers
1,TR16356,165,01/01/2022,P9,10000,7,70000,1,44,1,Married,"14,58",Prima Tendean,Prima,Modern Trade,"-6,2","106,816666",Yoghurt
2,TR1984,183,01/01/2022,P1,8800,4,35200,4,27,1,Single,"0,18",Gita Ginara,Gita,General Trade,"-6,966667","110,416664",Choco Bar
3,TR35256,160,01/01/2022,P1,8800,7,61600,4,48,1,Married,"12,57",Gita Ginara,Gita,General Trade,"-6,966667","110,416664",Choco Bar
4,TR41231,386,01/01/2022,P9,10000,1,10000,4,33,0,Married,"6,95",Gita Ginara,Gita,General Trade,"-6,966667","110,416664",Yoghurt


In [65]:
# Mengonversi kolom 'Income' ke numerik
df['Income'] = df['Income'].str.replace(',', '').astype(float)

# Mengisi nilai yang hilang pada 'Marital Status'
mode_value = df['Marital Status'].mode()[0]
df['Marital Status'].fillna(mode_value, inplace=True)

In [66]:
import pandas as pd

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Feature Engineering
df['ProductID_count'] = df.groupby('ProductID')['ProductID'].transform('count')
df['AmountPerProduct'] = df['TotalAmount'] / df['ProductID_count']
df['ProductID_nunique'] = df['ProductID'].nunique()
df['CustomerID_nunique'] = df['CustomerID'].nunique()
df['ProductToCustomerRatio'] = df['ProductID_nunique'] / df['CustomerID_nunique']
df['IncomeToAmount'] = df['Income'] / df['TotalAmount']
df['AmountPerQty'] = df['TotalAmount'] / df['Qty']
df['QtyToPrice'] = df['Qty'] / df['Price_x']

# Feature engineering on Date column
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
df['WeekOfYear'] = df['Date'].dt.weekofyear
df['Quarter'] = df['Date'].dt.quarter
df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)

# Aggregation for main features
aggregation_functions = {
    'TransactionID': ['count', 'nunique'],
    'CustomerID': ['count', 'nunique'],
    'ProductID': ['count', 'nunique'],
    'Age': ['mean', 'std', 'median', 'max', 'min'],
    'Income': ['mean', 'std', 'median', 'max', 'min'],
    'Qty': ['sum', 'mean'],
    'TotalAmount': ['sum', 'mean', 'std', 'max', 'min', 'median'],
    'Price_x': ['mean', 'std', 'median', 'max', 'min'],
    'AmountPerProduct': 'mean',
    'ProductToCustomerRatio': 'mean',
    'IncomeToAmount': 'mean',
    'AmountPerQty': 'mean',
    'QtyToPrice': 'mean',
    'Year': 'first',
    'Month': 'first',
    'Day': 'first',
    'DayOfWeek': 'first',
    'IsWeekend': 'mean',
    'WeekOfYear': 'first',
    'Quarter': 'first',
    'IsMonthStart': 'sum',
    'IsMonthEnd': 'sum'
}
aggregated_features = df.groupby('Date').agg(aggregation_functions).reset_index()
aggregated_features.columns = ['_'.join(col).strip() if col[1] else col[0] for col in aggregated_features.columns.values]

# One-hot encoding
ohe_columns = ['ProductID', 'Age', 'StoreID', 'StoreName', 'GroupStore', 'Type', 'Latitude', 'Longitude', 'Marital Status', 'Gender', 'Product Name']
ohe_features = pd.get_dummies(df[['Date'] + ohe_columns], columns=ohe_columns)
ohe_features = ohe_features.groupby('Date').sum().reset_index()

# # Merging features
# final_features = pd.concat([aggregated_features, ohe_features], axis=1)

# Merging features
final_features = pd.merge(aggregated_features, ohe_features, on='Date', how='left')

final_features['Date'] = pd.to_datetime(final_features['Date'], format='%d/%m/%Y')
final_features.sort_values(by='Date', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_7116\225755535.py:22: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['WeekOfYear'] = df['Date'].dt.weekofyear


In [67]:
# Renaming columns to avoid special characters
final_features.columns = final_features.columns.str.replace('[', '_')
final_features.columns = final_features.columns.str.replace(']', '_')
final_features.columns = final_features.columns.str.replace('{', '_')
final_features.columns = final_features.columns.str.replace('}', '_')
final_features.columns = final_features.columns.str.replace('<', '_')
final_features.columns = final_features.columns.str.replace('>', '_')
final_features.columns = final_features.columns.str.replace(':', '_')
final_features.columns = final_features.columns.str.replace(',', '_')


C:\Users\USER\AppData\Local\Temp\ipykernel_7116\3134267271.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_features.columns = final_features.columns.str.replace('[', '_')
C:\Users\USER\AppData\Local\Temp\ipykernel_7116\3134267271.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_features.columns = final_features.columns.str.replace(']', '_')
C:\Users\USER\AppData\Local\Temp\ipykernel_7116\3134267271.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_features.columns = final_features.columns.str.r

In [68]:
# Train/test split
from sklearn.model_selection import train_test_split

X = final_features.drop(['Qty_sum', 'Date'], axis=1)
y = final_features['Qty_sum']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
def smape(true, predicted):
    return 100 * np.mean(2 * np.abs(predicted - true) / (np.abs(true) + np.abs(predicted)))

# Fungsi untuk menghitung metrik
def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = mean_squared_error(actual, predicted, squared=False)
    mape = (100 * (abs(actual - predicted) / predicted)).mean()
    smape_value = smape(actual, predicted)
    r2 = r2_score(actual, predicted)
    
    return mae, mse, rmse, mape, smape_value, r2

In [70]:
# Membuat objek model
catboost_model = CatBoostRegressor(silent=True)
lgbm_model = LGBMRegressor()
rf_model = RandomForestRegressor()
knn_model = KNeighborsRegressor()
linear_model = LinearRegression()


In [71]:
# Misalkan X adalah fitur dan y adalah target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = [catboost_model, lgbm_model, rf_model, knn_model, linear_model]
model_names = ["CatBoost", "LGBM", "RandomForest", "KNN", "Linear Regression"]

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae, mse, rmse, mape, smape_value, r2 = calculate_metrics(y_test, predictions)
    print(f"Metrik untuk {name}:")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")
    print(f"R2: {r2}")
    print("-----")


Metrik untuk CatBoost:
MAE: 1.4496590276812698
MSE: 6.886898166303197
RMSE: 2.6242900309042057
MAPE: 2.770713626955874
SMAPE: 2.8064471946495266
R2: 0.9748932249657745
-----
Metrik untuk LGBM:
MAE: 2.18345394302876
MSE: 10.423550769689918
RMSE: 3.228552426349914
MAPE: 4.3398210542123135
SMAPE: 4.317178463745372
R2: 0.9620000560611003
-----
Metrik untuk RandomForest:
MAE: 2.3890410958904114
MSE: 10.514613698630138
RMSE: 3.2426245078069305
MAPE: 5.228428759376804
SMAPE: 5.231110015849297
R2: 0.9616680783817952
-----
Metrik untuk KNN:
MAE: 4.824657534246575
MSE: 40.845479452054796
RMSE: 6.391046819735777
MAPE: 10.407074146735066
SMAPE: 10.351553595102171
R2: 0.8510943186606903
-----
Metrik untuk Linear Regression:
MAE: 0.5668879982352785
MSE: 0.5447278746844266
RMSE: 0.7380568234793488
MAPE: 1.203569032041812
SMAPE: 1.2029616016313638
R2: 0.998014148042512
-----
